In [1]:
import numpy as np
import pickle
import sklearn as sk
from sklearn import svm
from sklearn import calibration
import matplotlib.pyplot as plt
from metrics import compute_precision_recall_events
import pandas as pd
import scipy as sp
import os
import csv

In [11]:
# Load data (deserialize)
Root='C:\ProyectoInicial\Otros_modelos\Test_comparisson'
csvfile=open(Root+'\Comparisson.csv', 'w')
# create the csv writer
writer = csv.writer(csvfile)
# write a row to the csv file
Best_models=[]
for filename in os.listdir(Root):
    f = os.path.join(Root, filename)
    print(filename)
    if filename!='SVM_train.pickle':
        continue
    # checking if it is a file

    with open(f, 'rb') as handle:
        Saved=(pickle.load(handle))
    F1_train=Saved['results']['performance'][-4]
    F1_test=Saved['results']['performance'][-1]
    # open the file in the write mode
    # Sólo se guardan en validación las que superen un determinado valor de train F1
    writer.writerow(['code','F1_train','F1_test','F1_val_Dlx1','F1_val_Thy7','F1_val_PV6','F1_val_PV7xChR2', 'F1_val_Thy9','F1_valThy1GCam1','F1_val_mean'])
    row=[Saved['params'],F1_train,F1_test,-1,-1,-1,-1,-1,-1,-1]
    writer.writerow(row)
    Val={
        "Code": Saved['params'],
        "F1 train": F1_train,
        "F1 test": F1_test,
        "F1 val ses Dlx1": -1,
        "F1 val ses Thy7": -1,
        "F1 val ses PV6": -1,
        "F1 val ses PV7xChR2": -1,
        "F1 val ses Thy9": -1,
        "F1 val ses Thy1GCam1": -1,
        "F1 val mean": -1,
        }
    Best_models.append(Val)
csvfile.close()

CNN2D_train.pickle
CNN2D_val.pickle
Comparisson.csv
DeepNN_val.pickle
LSTM_val.pickle
Prediction plot.ipynb
SVM_val.pickle
Validation plot.ipynb
XGBoost_train.pickle
XGBoost_val.pickle


In [10]:
def get_predictions_index(predictions,threshold=0.5):
    aux=np.copy(predictions)
    aux[aux>=threshold]=1
    aux[aux<threshold]=0
    pred_indexes =[]
    dif=np.diff(aux,axis=0)
    for i_pred,pred in enumerate(dif):
        if pred==1:
            pred_indexes.append(i_pred)
        elif pred==-1:
            pred_indexes.append(i_pred)
    if len(pred_indexes)%2==1:
        pred_indexes=pred_indexes[:-1]    
    pred_indexes=np.array(pred_indexes).reshape(-1,2)
    return(np.array(pred_indexes).reshape(-1,2))

In [16]:
n_sessions=6
th_arr=np.linspace(0.05,1,20)
results=np.empty(shape=(n_sessions,len(th_arr),5))
print(np.shape(results))
session={
        "0": "Dlx1",
        "1": "Thy7",
        "2": "PV6",
        "3": "PV7xChR2",
        "4": "Thy9",
        "5": "Thy1GCam1",
    }

window=1
signal_path="\\06_01_2022"
    # Carga del modelo que toque
with open('C:\ProyectoInicial\Otros_modelos\SVM\modelos\M_L_NW_US_01.pickle', 'rb') as handle:
    clf=pickle.load(handle)
for s in range (n_sessions):
    # Carga de los datos de validación (las 2 sesiones que no he utilizado para entrenar)
    with open('C:\ProyectoInicial\Datos_pickle\\x_'+session[str(s)]+'.pickle', 'rb') as handle:
        x=pickle.load(handle)
    with open('C:\ProyectoInicial\Datos_pickle\\y_'+session[str(s)]+'.pickle', 'rb') as handle:
        y=pickle.load(handle)
    print(np.shape(x))
    print(np.shape(y))
    # Predicción
    y_predict = clf.predict_proba(x)[:,1]
    with open('C:\ProyectoInicial\Otros_modelos\Salidas\SVM'+signal_path+'\\r_signal_'+session[str(s)] +'.pickle', 'wb') as handle:
        pickle.dump(y_predict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    y_gt_ind=get_predictions_index(y)
    for i,th in enumerate(th_arr):
        print(th)
        y_pred_ind=get_predictions_index(y_predict,th)
        prec,rec,F1,a,b,c=compute_precision_recall_events(y_pred_ind,y_gt_ind,0)
        # Modelo, # th1, #th2, P,R y F1
        results[s][i]=[s,th,prec, rec, F1]

# Otro for con cada sesión?
Validation_results={
    "Params":"SVM",
    "Performance":results,
}
with open(Root+ '\SVM_val.pickle', 'wb') as handle:
    pickle.dump(Validation_results, handle, protocol=pickle.HIGHEST_PROTOCOL)
# Rellenar el csv, hacer la media de las sesiones


(6, 20, 5)
(1276672, 8)
(1276672, 1)
0.05
precision = 0.008011864652684862
recall = 0.8199052132701422
F1 = 0.015868665526823235
0.1
precision = 0.0201534109137574
recall = 0.985781990521327
F1 = 0.03949929488119336
0.15
precision = 0.037179140772764116
recall = 0.9478672985781991
F1 = 0.07155173668960899
0.2
precision = 0.06188742649117894
recall = 0.909952606635071
F1 = 0.11589278715434208
0.25
precision = 0.09009419629292008
recall = 0.8720379146919431
F1 = 0.1633155242697448
0.3
precision = 0.12306610407876231
recall = 0.8341232227488151
F1 = 0.21448692012799578
0.35
precision = 0.1609907120743034
recall = 0.7677725118483412
F1 = 0.2661695472210812
0.39999999999999997
precision = 0.2193384223918575
recall = 0.7393364928909952
F1 = 0.3383105104394927
0.44999999999999996
precision = 0.28244837758112096
recall = 0.6729857819905214
F1 = 0.39790024326449985
0.49999999999999994
precision = 0.3282296650717703
recall = 0.6350710900473934
F1 = 0.4327810812464984
0.5499999999999999
precision

C:\Users\adrub\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\adrub\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0.05
precision = 0.08345666302039766
recall = 0.9885386819484241
F1 = 0.15391883938525336
0.1
precision = 0.15648226667331772
recall = 0.9551098376313276
F1 = 0.2689075457368918
0.15
precision = 0.23958985597996243
recall = 0.9226361031518625
F1 = 0.38039806173524904
0.2
precision = 0.3337810611148422
recall = 0.889207258834766
F1 = 0.48536938180589934
0.25
precision = 0.4153846153846154
recall = 0.8443170964660935
F1 = 0.556824411809306
0.3
precision = 0.48446511627906974
recall = 0.7917860553963706
F1 = 0.6011241860678173
0.35
precision = 0.5469651511654743
recall = 0.7459407831900668
F1 = 0.6311419916893176
0.39999999999999997
precision = 0.6010897619730428
recall = 0.6943648519579751
F1 = 0.6443693188437574
0.44999999999999996
precision = 0.6501558711465188
recall = 0.6380133715377269
F1 = 0.6440273927218981
0.49999999999999994
precision = 0.6958993097848153
recall = 0.5950334288443171
F1 = 0.6415258363830416
0.5499999999999999
precision = 0.7408491947291361
recall = 0.544412607449

C:\Users\adrub\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\adrub\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0.05
precision = 0.017341642870542136
recall = 0.9904761904761905
F1 = 0.03408648626503063
0.1
precision = 0.052102331542367035
recall = 0.9642857142857143
F1 = 0.09886289826706735
0.15
precision = 0.10481470074940971
recall = 0.9166666666666666
F1 = 0.18811922648509669
0.2
precision = 0.168115438108484
recall = 0.8571428571428572
F1 = 0.2810978416220537
0.25
precision = 0.22417355371900827
recall = 0.780952380952381
F1 = 0.3483521108838011
0.3
precision = 0.2914285714285714
recall = 0.6928571428571428
F1 = 0.4102840555670745
0.35
precision = 0.33879222108495394
recall = 0.611904761904762
F1 = 0.43611913593380314
0.39999999999999997
precision = 0.3892128279883382
recall = 0.5285714285714286
F1 = 0.44831185333091306
0.44999999999999996
precision = 0.4428997020854022
recall = 0.46190476190476193
F1 = 0.4522026351135942
0.49999999999999994
precision = 0.48855989232839836
recall = 0.3833333333333333
F1 = 0.42959685096092
0.5499999999999999
precision = 0.5525291828793775
recall = 0.32857142

...

C:\Users\adrub\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\adrub\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0.05
precision = 0.0038473548041752163
recall = 0.6174242424242424
F1 = 0.0076470585035671316
0.1
precision = 0.008689382314333784
recall = 0.7954545454545454
F1 = 0.017190973954890675
0.15
precision = 0.017990454044792558
recall = 0.9848484848484849
F1 = 0.03533542769550513
0.2
precision = 0.026949700046146746
recall = 0.9772727272727273
F1 = 0.052452935020772716
0.25
precision = 0.04015039613267087
recall = 0.9734848484848485
F1 = 0.07712005379325225
0.3
precision = 0.05676126878130217
recall = 0.9621212121212122
F1 = 0.10719827211678107
0.35
precision = 0.07420048034382505
recall = 0.9545454545454546
F1 = 0.13769722695411768
0.39999999999999997
precision = 0.09292328042328042
recall = 0.9393939393939394
F1 = 0.16911771843482143
0.44999999999999996
precision = 0.11319148936170213
recall = 0.9204545454545454
F1 = 0.20159245501922737
0.49999999999999994
precision = 0.13585209003215434
recall = 0.9128787878787878
F1 = 0.23650775216305678
0.5499999999999999
precision = 0.1604696673189823

C:\Users\adrub\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\adrub\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0.05
precision = 0.019099316445516688
recall = 0.9958333333333333
F1 = 0.03747979920533912
0.1
precision = 0.038189291014956044
recall = 0.9791666666666666
F1 = 0.07351149910340425
0.15
precision = 0.06552494415487714
recall = 0.9625
F1 = 0.12269694253560402
0.2
precision = 0.09244629244629245
recall = 0.9416666666666667
F1 = 0.1683637967911452
0.25
precision = 0.12164987644934423
recall = 0.9208333333333334
F1 = 0.21490851877372685
0.3
precision = 0.15733922434953362
recall = 0.9125
F1 = 0.26839928645585376
0.35
precision = 0.19310132429935326
recall = 0.9041666666666667
F1 = 0.3182372622880459
0.39999999999999997
precision = 0.23082515448927662
recall = 0.8791666666666667
F1 = 0.36564914765555817
0.44999999999999996
precision = 0.275621456607065
recall = 0.85
F1 = 0.41626469847542663
0.49999999999999994
precision = 0.31262729124236255
recall = 0.8
F1 = 0.44956983342485823
0.5499999999999999
precision = 0.34408602150537637
recall = 0.7791666666666667
F1 = 0.47736428542938064
0.6
preci

C:\Users\adrub\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\adrub\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [17]:
with open('C:\ProyectoInicial\Otros_modelos\SVM\SVM_val.pickle', 'wb') as handle:
    pickle.dump(Validation_results, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('C:\ProyectoInicial\Otros_modelos\Test_comparisson\SVM_val.pickle', 'wb') as handle:
    pickle.dump(Validation_results, handle, protocol=pickle.HIGHEST_PROTOCOL)
# Rellenar el csv, hacer la media de las sesiones
